In [1]:
#  Add root folder to python path
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np 
import pandas as pd
import scipy.stats as ss

from pof.data.asset_data import SimpleFleet

In [3]:
# Forecast years
START_YEAR = 2015
END_YEAR = 2024
CURRENT_YEAR = 2020

# Population Data
file_path = r"C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\inputs" + os.sep
FILE_NAME = r"population_summary.csv"

sfd = SimpleFleet(file_path + FILE_NAME)
sfd.load()
sfd.calc_age_forecast(START_YEAR, END_YEAR, CURRENT_YEAR)
df_age_forecast = sfd.df_forecast
df_age_forecast

## Check if it plays well with component

In [4]:
from pof import Component

In [5]:
t_end = 100
n_iterations = 100

comp = Component.demo()
comp.mc_timeline(t_end=t_end, n_iterations=n_iterations)

100%|██████████| 100/100 [00:06<00:00, 15.54it/s]


In [6]:
df_erc = comp.expected_risk_cost_df()
df_erc

,failure_mode,task,active,time,quantity,cost,cost_cumulative,cost_annual,quantity_cumulative,quantity_annual
12,early_life,risk,True,0,0,0,0,0,0,0
13,random,risk,True,0,0,0,0,0,0,0
14,slow_aging,risk,True,0,0,0,0,0,0,0
15,fast_aging,risk,True,0,0,0,0,0,0,0
0,early_life,inspection,True,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
7,fast_aging,on_condition_replacement,True,99,0,0,350,0,0.07,0
8,early_life,on_failure_replacement,True,99,0,0,300,0,0.03,0
9,random,on_failure_replacement,True,99,0,0,700,0,0.07,0
10,slow_aging,on_failure_replacement,True,99,0,0,900,0,0.09,0


In [7]:
df_task_forecast = comp.calc_df_task_forecast(sfd.df_age_forecast)
df_task_forecast

,year,task,active,pop_quantity,pop_cost
0,2015,risk,True,42.95,21475.000
1,2015,inspection,True,116195.42,5470212.394
2,2015,on_condition_repair,True,1850.45,11007.230
3,2015,on_condition_replacement,True,0.00,0.000
4,2015,on_failure_replacement,True,42.95,4295.000
5,2016,risk,True,638.44,734230.000
6,2016,inspection,True,122307.92,6102088.465
7,2016,on_condition_repair,True,486.48,1459.440
8,2016,on_condition_replacement,True,0.00,0.000
9,2016,on_failure_replacement,True,638.44,146846.000


In [8]:
df_summary = comp.calc_summary(sfd.df_age)
df_summary

,fm,ie (%),is,cf,ff,sim (%),cf_pop_annual_avg,ff_pop_annual_avg,total
0,early_life,18,97,0,3,NaN,0,256.82,NaN
1,random,18,93,0,7,NaN,0,778.64,NaN
2,slow_aging,95.4,89,2,9,18.1818,56.9,158.04,NaN
3,fast_aging,95.4,71,6,23,20.6897,227.06,2892.43,NaN
4,total,2.94878,50,8,42,16,283.96,4085.93,1174929.0


In [15]:
comp.plot_ms().show()
comp.plot_task_forecast().show()
comp.plot_summary().show()

## Check it scales with times

In [44]:
from pof.units import unit_ratio
model_units = 'years'
input_units = 'years'
t_end = 200 * int(unit_ratio(model_units, input_units))

comp.units = model_units
comp.mc_timeline(t_end=t_end, n_iterations=n_iterations)
comp.expected_risk_cost_df()
comp.calc_df_task_forecast(sfd.df_age_forecast)

# Display plots
comp.plot_ms().show()
comp.plot_task_forecast().show()
comp.plot_summary().show()

100%|██████████| 100/100 [00:09<00:00, 10.51it/s]


In [29]:
comp.fm['fast_aging'].pf_interval

0.0

In [48]:
comp2 = Component.demo()
comp3 = Component.demo()
comp2.units = 'months'
#comp2.units='years'


comp2.indicator['slow_degrading'].__dict__
comp2.fm['slow_aging'].tasks['inspection'].__dict__


{'_name': 'inspection',
 '_units': 'months',
 'up_to_date': True,
 'task_type': 'Inspection',
 'task_category': 'Graphing purposes',
 'task_group_name': 'unknown',
 'trigger': 'time',
 'active': True,
 '_package': NotImplemented,
 '_impacts_parent': NotImplemented,
 '_impacts_children': False,
 'cost': 55,
 'labour': NotImplemented,
 'spares': NotImplemented,
 'equipment': NotImplemented,
 'consequence': <pof.consequence.Consequence at 0x21496150a90>,
 'p_effective': 0.9,
 'triggers': {'condition': {'fast_degrading': {'lower': 0, 'upper': 90},
   'slow_degrading': {'lower': 0, 'upper': 90}},
  'state': {'initiation': True},
  'time': {}},
 'impacts': {'condition': {},
  'state': {'detection': True},
  'time': {},
  'system': []},
 'state': NotImplemented,
 't_completion': [],
 'cost_completion': [],
 '_timeline': NotImplemented,
 '_t_interval': 60.0,
 '_t_delay': 120.0}

In [16]:
scale_units(
            comp.df_erc, "years", comp.units
        )[0]

,failure_mode,task,active,time,quantity,cost,cost_cumulative,cost_annual,quantity_cumulative,quantity_annual
12,early_life,risk,True,0,0,0,0,0,0,0
13,random,risk,True,0,0,0,0,0,0,0
14,slow_aging,risk,True,0,0,0,0,0,0,0
15,fast_aging,risk,True,0,0,0,0,0,0,0
0,early_life,inspection,True,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
7,fast_aging,on_condition_replacement,True,1.91781,0,0,0,0,0,0
8,early_life,on_failure_replacement,True,1.91781,0,0,0,0,0,0
9,random,on_failure_replacement,True,1.91781,0,0,0,0,0,0
10,slow_aging,on_failure_replacement,True,1.91781,0,0,0,0,0,0


In [14]:
left = pd.DataFrame({"a": [1, 5, 10], "left_val": ["a", "b", "c"]})
right = pd.DataFrame({"a": [1, 2, 3, 6, 7], "right_val": [1, 2, 3, 6, 7]})
pd.merge_asof(right, left, on="a")

,a,right_val,left_val
0,1,1,a
1,2,2,a
2,3,3,a
3,6,6,b
4,7,7,b


In [14]:
comp.df_erc

,failure_mode,task,active,time,quantity,cost,cost_cumulative,cost_annual,quantity_cumulative,quantity_annual
12,early_life,risk,True,0,0,0,0,0,0,0
13,random,risk,True,0,0,0,0,0,0,0
14,slow_aging,risk,True,0,0,0,0,0,0,0
15,fast_aging,risk,True,0,0,0,0,0,0,0
0,early_life,inspection,True,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
7,fast_aging,on_condition_replacement,True,100,0,0,0,0,0,0
8,early_life,on_failure_replacement,True,100,0,0,0,0,0,0
9,random,on_failure_replacement,True,100,0,0,0,0,0,0
10,slow_aging,on_failure_replacement,True,100,0,0,0,0,0,0


In [31]:
sfd.df_age

,assets
age,
0,7124
2,9817
4,8745
6,10699
8,11583
...,...
240,5
276,1
280,1


In [22]:
scale_units(sfd.df_age.reset_index(), 'months', 'years')[0]

,age,assets
0,0.0,7124
1,12.0,9817
2,24.0,8745
3,36.0,10699
4,48.0,11583
...,...,...
106,1440.0,5
107,1656.0,1
108,1680.0,1
109,1740.0,1
